In [3]:
import cufflinks as cf

import plotly_express as px

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


from pymongo import MongoClient

from beakerx import *
from beakerx.object import beakerx

init_notebook_mode(connected=True)
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

beakerx.pandas_display_table()
client = MongoClient()

# This notebook shows the visualization of data transformation experiments

## We retrieve Active Learning Accuracy (ALC) between IPBE and UDATA from result database.

In [4]:
import itertools
dataset_wth_alc = []

for dataset in ["ijcai", "sygus", "museum", "nyc", "prog"]:
     dataset_wth_alc.extend([
        {
            "method": x["config"]["mapping_method"],
            "acc": x["result"]["active_learning_curve"],
            "dataset": dataset,
        }
        for x in client.sacred.runs.find({"config.dataset": dataset})
    ])


df = pd.DataFrame(dataset_wth_alc)
df

TypeError: 'NoneType' object is not subscriptable

In [ ]:
df2 = df.set_index(["dataset", "method"])["acc"].apply(pd.Series).stack()
df2 = df2.reset_index().drop_duplicates()
df2.columns = ["dataset", "method", "pos", "acc"]
px.line(df2, x="pos", y="acc", facet_row="dataset", color="method", height=1500)